<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/sentence_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install datasets transformers sentence-transformers

     |████████████████████████████████| 325 kB 33.8 MB/s 
     |████████████████████████████████| 4.0 MB 47.7 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 136 kB 70.8 MB/s 
     |████████████████████████████████| 212 kB 72.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.2 MB/s 
     |████████████████████████████████| 77 kB 9.2 MB/s 
     |████████████████████████████████| 127 kB 76.8 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
     |████████████████████████████████| 895 kB 63.2 MB/s 
     |████████████████████████████████| 1.2 MB 60.4 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 76.7 MB/s 
     |████████████████████████████████| 144 kB 71.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=50bb1c6e8057db7cd739

In [2]:
import datasets

dataset=datasets.load_dataset("glue","mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
for item in dataset["test"].select(range(20)):
    print(item)

{'sentence1': "PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .", 'sentence2': 'Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .', 'label': 1, 'idx': 0}
{'sentence1': "The world 's two largest automakers said their U.S. sales declined more than predicted last month as a late summer sales frenzy caused more of an industry backlash than expected .", 'sentence2': 'Domestic sales at both GM and No. 2 Ford Motor Co. declined more than predicted as a late summer sales frenzy prompted a larger-than-expected industry backlash .', 'label': 1, 'idx': 1}
{'sentence1': 'According to the federal Centers for Disease Control and Prevention ( news - web sites ) , there were 19 reported cases of measles in the United States in 2002 .', 'sentence2': 'The Centers for Disease Control and Prevention said there were 19 reported cases of measles in the United States in 2

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')


Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

In [5]:
help(model.encode)

Help on method encode in module sentence_transformers.SentenceTransformer:

encode(sentences: Union[str, List[str]], batch_size: int = 32, show_progress_bar: bool = None, output_value: str = 'sentence_embedding', convert_to_numpy: bool = True, convert_to_tensor: bool = False, device: str = None, normalize_embeddings: bool = False) -> Union[List[torch.Tensor], numpy.ndarray, torch.Tensor] method of sentence_transformers.SentenceTransformer.SentenceTransformer instance
    Computes sentence embeddings
    
    :param sentences: the sentences to embed
    :param batch_size: the batch size used for the computation
    :param show_progress_bar: Output a progress bar when encode sentences
    :param output_value:  Default sentence_embedding, to get sentence embeddings. Can be set to token_embeddings to get wordpiece token embeddings. Set to None, to get all output values
    :param convert_to_numpy: If true, the output is a list of numpy vectors. Else, it is a list of pytorch tensors.
    :p

In [9]:
#At first, the run was very slow
#the way to debug: notice in colab GPU resources that GPU memory is 0
#i.e. the model needs to be pushed to the GPU
model=model.cuda()

In [6]:
texts1=[item["sentence1"] for item in dataset["test"].select(range(1000))]
texts2=[item["sentence2"] for item in dataset["test"].select(range(1000))]

In [13]:
texts1_e=model.encode(texts1,convert_to_tensor=True,device=model.device,show_progress_bar=True,normalize_embeddings=True)
texts2_e=model.encode(texts2,convert_to_tensor=True,device=model.device,show_progress_bar=True,normalize_embeddings=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [36]:
import torch
sims=torch.mm(texts1_e,texts2_e.T)
sims.shape

torch.Size([1000, 1000])

In [37]:
sims_sort=torch.argsort(-sims,dim=-1)

In [38]:
sims_sort[:,0]

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8, 668,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34, 896,  36,  37,  38,  39,  40,  41,
         42,  43,  44, 913,  46, 778,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74, 951,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 666, 932, 107, 108, 109, 110, 379,
        112, 113, 114, 115, 116, 345, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 698, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 794, 176, 804, 178, 1

In [28]:
print(texts1[5])
print(texts2[996])

The settling companies would also assign their possible claims against the underwriters to the investor plaintiffs , he added .
The 2 1 / 2-ton probe will plunge into the thick Jovian atmosphere today at 1 : 49 p.m. MDT , disintegrating moments later from the friction generated by its 108,000 mph free-fall .
